In [1]:
NOTE_DATA_DIR = '/local/XW/DATA/MIMIC/noteevents_by_sid/'

In [2]:
import os,sys, re
import numpy as np
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproductive


In [3]:
from stop_words import get_stop_words
en_stop = get_stop_words('en')
en_stop = set(en_stop)

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|_') # underscore can't be matched by \W+

from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

import gensim
from gensim import corpora, models

import re
filters='(\W)|(\d+[a-z]?)'
pat = re.compile(filters)

In [ ]:
notes = []
for fn in tqdm(os.listdir(NOTE_DATA_DIR)):
    fpath = os.path.join(NOTE_DATA_DIR, fn)
    df = pd.read_csv(fpath)    
    for note in df.text:
        notes.append(note.lower())

100%|██████████| 46146/46146 [01:55<00:00, 398.76it/s]


In [16]:

# re.sub(filters, '', note)

In [ ]:
texts = []
for raw_note in notes:
    no_stop = filter( lambda tk: tk not in en_stop, tokenizer.tokenize(raw_note) )
    filtered = filter(lambda tk: pat.match(tk) is None and len(tk)>1, no_stop)
    stemmed = map(lambda tk: p_stemmer.stem(tk), filtered)
    texts.append( stemmed )

In [ ]:
print texts[0]

In [ ]:
dictionary = corpora.Dictionary(texts)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts] # convert to (token_id, token_cnt) tuple

In [ ]:
# %timeit -n1 -r1 ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=300, id2word = dictionary, passes=20)

In [ ]:
%timeit -n1 -r1 ldamodel = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics=300, batch=True, passes=1,\
                                                                   id2word=dictionary, workers=3)

In [ ]:
ldamodel.print_topics(20)